In [2]:
!git clone https://github.com/megapose6d/megapose6d.git
%cd megapose6d
!git submodule update --init

# Install megapose using pip directly
!pip install -e.

# Modify the scripts to use sys.executable instead of CONDA_PREFIX
import sys
import fileinput

scripts_to_modify = [
    'src/megapose/scripts/download.py',
    'src/megapose/scripts/run_inference_on_example.py',
    'src/megapose/config.py' # Modify config.py as well
]

for script_path in scripts_to_modify:
    with fileinput.FileInput(script_path, inplace=True, backup='') as file:
        for line in file:
            if script_path == 'src/megapose/config.py' and 'PYTHON_BIN_PATH = Path(sys.executable)' in line:
                print('import sys')
                print(line, end='')
            # Replace the line that uses os.environ["CONDA_PREFIX"]
            elif 'PYTHON_BIN_PATH = Path(os.environ["CONDA_PREFIX"])' in line:
                print('PYTHON_BIN_PATH = Path(sys.executable)')
            else:
                print(line, end='')

print("Hello")
!python -m megapose.scripts.download --megapose_models
!python -m megapose.scripts.run_inference_on_example barbecue-sauce --run-inference

Cloning into 'megapose6d'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 205 (delta 23), reused 14 (delta 14), pack-reused 146 (from 2)
Receiving objects: 100% (205/205), 19.55 MiB | 15.64 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/megapose6d/megapose6d
Submodule 'deps/bop_toolkit_challenge' (https://github.com/ylabbe/bop_toolkit_challenge) registered for path 'deps/bop_toolkit_challenge'
Cloning into '/content/megapose6d/megapose6d/deps/bop_toolkit_challenge'...
Submodule path 'deps/bop_toolkit_challenge': checked out '55f3bf1f70d7a85e846e747cde19633117bfe0df'
Obtaining file:///content/megapose6d/megapose6d
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for megapose (pyproject.toml) ... 

In [5]:
!ls -R /content/megapose6d/

/content/megapose6d/:
CLA    docker	license_files  notebooks       README.md  src
conda  images	local_data     pyproject.toml  setup.cfg
deps   LICENSE	megapose6d     rclone.conf     setup.py

/content/megapose6d/conda:
environment_full.yaml  environment.yaml

/content/megapose6d/deps:
bop_toolkit_challenge

/content/megapose6d/deps/bop_toolkit_challenge:
bop_toolkit_lib  docs  LICENSE	README.md  requirements.txt  scripts  setup.py

/content/megapose6d/deps/bop_toolkit_challenge/bop_toolkit_lib:
colors.json		     inout.py	       renderer_py.py
config.py		     misc.py	       score.py
dataset_params.py	     pose_error.py     transform.py
droid_sans_mono_license.txt  pose_matching.py  view_sampler.py
droid_sans_mono.ttf	     renderer_cpp.py   visibility.py
__init__.py		     renderer.py       visualization.py

/content/megapose6d/deps/bop_toolkit_challenge/docs:
bop_challenge_2019_re-evaluation_27-01-2020.md	bop_datasets_format.md

/content/megapose6d/deps/bop_toolkit_challenge/scripts:
ca

In [7]:
!python -m megapose.scripts.run_inference_on_example barbecue-sauce --run-inference

MKL_NUM_THREADS: 1
OMP_NUM_THREADS: 1
CUDA_VISIBLE_DEVICES: 0
EGL_VISIBLE_DEVICES: 0
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/megapose6d/megapose6d/megapose6d/megapose6d/src/megapose/scripts/run_inference_on_example.py", line 15, in <module>
    from megapose.config import LOCAL_DATA_DIR
  File "/content/megapose6d/megapose6d/megapose6d/megapose6d/src/megapose/config.py", line 45, in <module>
    PYTHON_BIN_PATH = Path(sys.executable)
                           ^^^
NameError: name 'sys' is not defined. Did you forget to import 'sys'?


In [17]:
import os
import copy
import time
import torch
import numpy as np
import random
import transforms3d


from omegaconf import OmegaConf

import megapose

from megapose.datasets.datasets_cfg import make_scene_dataset
from megapose.config import LOCAL_DATA_DIR, NB_DATA_DIR
from megapose.training.utils import RGB_DIMS
from megapose.inference.utils import make_cameras
import pickle as pkl
from bokeh.io import show, output_notebook; output_notebook()
from megapose.visualization.bokeh_plotter import BokehPlotter
from bokeh.plotting import gridplot
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from pathlib import Path
import pandas as pd
from PIL import Image
import scipy


from megapose.training.utils import cast_images, cast_to_numpy, CudaTimer
from megapose.lib3d.camera_geometry import get_K_crop_resize
from megapose.datasets.scene_dataset import SceneObservation

from megapose.inference.pose_estimator import PoseEstimator, ObservationTensor
from megapose.inference.icp_refiner import ICPRefiner
from megapose.visualization.utils import adjust_brightness, tensor_image_to_uint8, \
get_ds_info, make_contour_overlay
from megapose.utils import transform_utils
from megapose.lib3d.cosypose_ops import (
    TCO_init_from_boxes,
    TCO_init_from_boxes_zup_autodepth,
    TCO_init_from_boxes_autodepth_with_R
)
from megapose.panda3d_renderer.types import Panda3dLightData


import matplotlib.pyplot as plt
%matplotlib inline

BRIGHTNESS_FACTOR=1.5

# zmq_url = "tcp://127.0.0.1:6000"
# zmq_url = "tcp://127.0.0.1:6001"
# zmq_url = "tcp://127.0.0.1:6004"

ModuleNotFoundError: No module named 'megapose'

In [ ]:
def get_scene_data(scene_ds, scene_id, view_id):
    df = scene_ds.frame_index
    x = df[(df.scene_id == scene_id) & (df.view_id==view_id)]
    ds_idx = x.iloc[0].name
    scene_data = scene_ds[ds_idx]
    return scene_data


def orthogonalize_rotation(T):
    rot = scipy.spatial.transform.Rotation.from_matrix(T[:3,:3])
    T[:3,:3] = rot.as_matrix()
    return T

In [ ]:
object_label = None

ds_name = 'ycbv'
scene_ds_name = f"{ds_name}.test"
n_refiner_iterations = 5


scene_id, im_idx, object_label = 54, 1, 'obj_000015' # drill
# scene_id, im_idx, object_label = 54, 1, 'obj_000003' # sugargox


view_id = im_idx

## Load data and visualize image

In [ ]:
# Load the images/data
scene_ds_kwargs = {'load_depth': True}
scene_ds = make_scene_dataset(scene_ds_name, **scene_ds_kwargs)
scene_data = get_scene_data(scene_ds, scene_id, view_id)

if scene_data.depth is not None:
    depth = torch.as_tensor(scene_data.depth).unsqueeze(-1)
    rgb = torch.as_tensor(scene_data.rgb)
    image = torch.cat([rgb, depth], dim=-1).numpy()
else:
    image = scene_data.rgb.numpy()

images = [image]
cameras = make_cameras([scene_data.camera_data])

plotter = BokehPlotter()
image_f = plotter.plot_image(images[0][...,RGB_DIMS].astype(np.uint8))
show(image_f)


if object_label is None:
    object_labels = None
else:
    object_labels = [object_label]
data = SceneObservation.collate_fn([scene_data], object_labels=object_labels)
observation_tensor = ObservationTensor.from_numpy(scene_data.rgb, depth=scene_data.depth, K=scene_data.camera_data.K)
observation_tensor = observation_tensor.cuda()


# Filter gt_detections to only keep the object we are interested in
gt_detections = data['gt_detections']
# Filter and only run the estimator for that object
df = gt_detections.infos
df = df[df.label == object_label]
detection_idx = df.iloc[0].name
gt_detections = gt_detections[[detection_idx]]
gt_detections = gt_detections.cuda()

## Load the model

Select whether to load a depth refiner or not and what type

In [ ]:
import megapose.inference.utils
cfg = OmegaConf.create()
cfg.model_name = 'sn-gso-4views-normals'
cfg.ds_name = ds_name
cfg.use_icp = False
depth_multiplier = None
per_iter_depth_multiplier = None
model_data = megapose.inference.utils.load_named_model(cfg)
result_name = cfg.model_name



refiner_model = model_data['refiner_model']
coarse_model = model_data['coarse_model']
obj_ds_name = model_data['obj_ds_name']
detector_model = model_data['detector_model']
renderer = refiner_model.renderer

mesh_db = refiner_model.mesh_db


depth_refiner = None
depth_refiner_type = None
# depth_refiner_type = "icp"
# depth_refiner_type = "teaser++"

if depth_refiner_type == "icp":
    depth_refiner = ICPRefiner(mesh_db, renderer)
elif depth_refiner_type == "teaserpp":
    from megapose.inference.teaserpp_refiner import TeaserppRefiner
    depth_refiner = TeaserppRefiner(mesh_db, renderer)

pose_estimator = PoseEstimator(refiner_model=refiner_model,
                              coarse_model=coarse_model,
                               detector_model=detector_model,
                               depth_refiner=depth_refiner,
                               bsz_objects=16,
                               bsz_images=576,
                              )



## Run Model Inference

- We perform the individual steps (detector, coarse, refiner, scoring) etc. separately to make the inference pipeline transparent. You can simply use pose_estimator.run_inference_pipeline to run them all at once.
- You can set the options as to whether to use the ground-truth detections or the detections from Mask-RCNN.
- Note: If you aren't using gt_detections and there are multiple object instances in the scene this won't work properly.
- If you are getting CUDA out of memory errors decrease `bsz_objects` and `bsz_images` to smaller values.

In [ ]:
# Options for inference
use_gt_detections = True # Note, if you aren't using gt_detections then this should be false
n_refiner_iterations = 5
n_pose_hypotheses = 5
return_debug_data = True
detection_filter_kwargs = {'labels': [object_label]}
run_depth_refiner = False


bsz_images = 128
bsz_objects = 2


pose_estimator.bsz_objects = bsz_objects
pose_estimator.bsz_images = bsz_images

# set the random seed
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


start_time = time.time()
with torch.no_grad():

    if use_gt_detections:
        detections = gt_detections
    else:
        # Only keep the top detection in each image
        detections = pose_estimator.forward_detection_model(observation_tensor, one_instance_per_class=True)

    # Filter and only run the estimator for that object

    detections = megapose.inference.utils.filter_detections(detections, **detection_filter_kwargs)
    detections = megapose.inference.utils.add_instance_id_to_detections(detections)
    detections = detections.cuda()


#     print("detections\n", detections)

    # We have split the inference into it's component steps for clarity. This is a copy of
    # what is in the pose_estimator.run_pipeline method
    # Run the coarse estimator using gt_detections
    data_TCO_coarse, extra_data = pose_estimator.forward_coarse_model(observation=observation_tensor,
                                       detections=detections, cuda_timer=True)

    print(f"Forward Coarse: total={extra_data['time']:.2f}, "\
          f"model_time={extra_data['model_time']:.2f}, render_time={extra_data['render_time']:.2f}")

    # Extract top-K coarse hypotheses
    data_TCO_filtered = pose_estimator.filter_pose_estimates(data_TCO_coarse,
                                                             top_K=n_pose_hypotheses,
                                                             filter_field='coarse_logit')

    # Refine the top_K coarse hypotheses
    preds, extra_data = pose_estimator.forward_refiner(observation_tensor, data_TCO_filtered,
                                                   n_iterations=n_refiner_iterations, keep_all_outputs=True)

    print(f"Refiner time: {extra_data['time']:.2f}")
    data_TCO_refined = preds[f'iteration={n_refiner_iterations}']
    refiner_preds = preds
    refiner_outputs = extra_data['outputs']

    # Score the refined poses using the coarse model.
    data_TCO_scored, extra_data = pose_estimator.forward_scoring_model(observation_tensor, data_TCO_refined)

    # Extract the highest scoring pose estimate for each instance_id
    data_TCO_final = pose_estimator.filter_pose_estimates(data_TCO_scored, top_K=1, filter_field='pose_logit')


    if run_depth_refiner:
        print("\n\n")
        t = time.time()
        data_TCO_depth_refiner, _ = pose_estimator.run_depth_refiner(observation_tensor, data_TCO_final,
                                                                    )
        depth_refiner_time = time.time() - t
    else:
        data_TCO_depth_refiner = None


elapsed = time.time() - start_time
print(f"Entire pose estimation pipeline took {elapsed:.2f} seconds")

print("Final Pose Estimate\n")
print(data_TCO_final)


## Run the entire pipeline

- The cell below shows how to run the entire pipeline in one function call, rather than each step individually.
- It is disabled by default, set the flag to `True` to run the function.

In [ ]:
if False:
    use_gt_detections = False
    if use_gt_detections:
        detections_in = gt_detections.cuda()
        run_detector=False
    else:
        detections_in = None
        run_detector=True


    detection_filter_kwargs = {'labels': [object_label], 'one_instance_per_class':True}

    data_TCO_out, pred_data = pose_estimator.run_inference_pipeline(observation_tensor,
                                         detections=detections_in,
                                         run_detector=run_detector,
                                         n_refiner_iterations=5,
                                         n_pose_hypotheses=5,
                                        detection_filter_kwargs=detection_filter_kwargs,
                                        cuda_timer=True)

    print(f"Inference pipeline: {pred_data['timing_str']}")
    print(f"Coarse model: {pred_data['coarse']['data']['timing_str']}")

## Extract data from refiner iterations

In [13]:
def plot_cosypose(data, data_TCO_final, refiner_outputs_in, object_label,
                  plot_iter=[1, 2, 3, 4], scene_data=None):
#     orig_renderer = object_predictor.pose_predictor.refiner_model.renderer
#     object_predictor.pose_predictor.refiner_model.renderer = renderer

    rows = []
    outputs = []

    df = data_TCO_final.infos
    df_filter = df[df.label == object_label]
    assert len(df_filter) == 1, f"There was more than one object named {object_label} in refiner_preds"

    refiner_batch_idx = df_filter.iloc[0]['refiner_batch_idx']
    refiner_instance_idx = df_filter.iloc[0]["refiner_instance_idx"]


    df_gt = data['gt_detections'].infos
    df_gt_filter = df_gt[df_gt.label == object_label]

    assert len(df_gt_filter) == 1, f"There was more than one object named {object_label} in data['gt_detections']"
    obj_idx_gt = df_gt_filter.iloc[0].name
    TWC = scene_data.camera_data.TWC.matrix
    TCO_gt = data['gt_detections'].poses[obj_idx_gt].cpu().numpy().astype(np.float64)
    TOC_gt = np.linalg.inv(TCO_gt)


    if 'data_TCO_init' in all_preds:
        data_TCO_init = all_preds['data_TCO_init']
        df = data_TCO_init.infos
        df = df[df.label == object_label]
        idx_tmp = df.index[0]
        TCO_coarse_init = cast_to_numpy(data_TCO_init.poses[idx_tmp], np.float64)
    else:
        TCO_coarse_init = None



    for n in plot_iter:
        refiner_outputs_iter = refiner_outputs_in[refiner_batch_idx][f'iteration={n}']
        image_crop = refiner_outputs[refiner_batch_idx][f'iteration={n}']['images_crop']\
            [refiner_instance_idx][RGB_DIMS]
        render_crop = refiner_outputs[refiner_batch_idx][f'iteration={n}']['renders']\
            [refiner_instance_idx][RGB_DIMS]

        image_crop = (image_crop.permute(1, 2, 0) * 255).cpu().numpy().astype(np.uint8)
        render_crop = (render_crop.permute(1, 2, 0) * 255).cpu().numpy().astype(np.uint8)

        image_f = plotter.plot_image(image_crop)
        render_f = plotter.plot_image(render_crop)
        overlay_f = plotter.plot_overlay(image_crop, render_crop)
        row = [image_f, render_f, overlay_f]
        TCO_pred = refiner_outputs[refiner_batch_idx][f'iteration={n}']['TCO_input'][refiner_instance_idx].cpu().numpy()
        TCO_output = refiner_outputs[refiner_batch_idx][f'iteration={n}']['TCO_input'][refiner_instance_idx].cpu().numpy()


        # compute errors
        TCO_pred = orthogonalize_rotation(TCO_pred)
        TOgt_O = np.linalg.inv(TCO_gt) @ TCO_pred
        TOgt_O = orthogonalize_rotation(TOgt_O)
        trans_err = np.linalg.norm(TOgt_O[:3,3])


        # Compute coarse score
        rgb = data['rgb']
        depth = data['depth']

        # [B,C,H,W], C=3 or 4 depending on if depth was empty or not
        # Compute score from coarse model
        images = cast_images_to_tensor(rgb, depth)
        K = data['cameras'].K.cuda().float()
        label = [object_label]
        TCO_pred_tensor = torch.tensor(TCO_pred).cuda().unsqueeze(0)
        out_ = coarse_model.forward_coarse(images, K, label, TCO_input=TCO_pred_tensor,
                                           return_debug_data=True)

        coarse_out = out_


        try:
            _, rot_err_angle_radians = transforms3d.axangles.mat2axangle(TOgt_O[:3,:3])
            rot_err_deg = np.rad2deg(np.abs(rot_err_angle_radians))
        except ValueError:
            print("got error while computing angle distance")
            rot_err_deg = -1




        infos = dict(figures=row,
                     TCO_output=TCO_output,
                     TCO_input=TCO_pred,
                     TCO_gt=TCO_gt,
                     TOC_gt=TOC_gt,
                     TOgt_O=TOgt_O,
                     label=object_label,
                     refiner_batch_idx=refiner_batch_idx,
                     refiner_instance_idx=refiner_instance_idx,
                     iteration=n,
                     refiner_outputs=refiner_outputs_iter,
                     scene_data=scene_data,
                     input_rgb_dims=copy.copy(refiner_model.input_rgb_dims),
                     input_depth_dims=copy.copy(refiner_model.input_depth_dims),
                     render_rgb_dims=copy.copy(refiner_model.render_rgb_dims),
                     render_depth_dims=copy.copy(refiner_model.render_depth_dims),
                     TCO_coarse_init=TCO_coarse_init,
                     trans_err=trans_err,
                     rot_err=rot_err_deg,
                     coarse_out=coarse_out,
#                      TCO_coarse_init=TCO_coarse_init,
#                      object_predictor_data=object_predictor_data,
                    )
        outputs.append(infos)

    return outputs




plot_iter = [1, 2, 3,4,5,6]
plot_iter = list(range(1, n_refiner_iterations+1))
# plot_iter = [1,2,3,4,5,6,7,8]
all_preds = preds
all_infos = plot_cosypose(data, data_TCO_final, refiner_outputs,
                          object_label, plot_iter, scene_data=scene_data)

for info in all_infos:
    info['result_name'] = result_name
all_infos = pd.DataFrame(all_infos)

# save_path = NB_DATA_DIR / f'{result_name}_ds_name={ds_name}_scene_id={scene_id}_im={view_id}_object_label={object_label}.pkl'
# save_path.write_bytes(pkl.dumps(all_infos.drop(columns=('figures'))))
# print("wrote", save_path)

NameError: name 'n_refiner_iterations' is not defined

## Make contour overlay figure

Overlay ground-truth and estimated pose

In [12]:
SAVE_DIR = NB_DATA_DIR/'figures'
SAVE_DIR.mkdir(parents=True, exist_ok=True)


ambient_light_data = Panda3dLightData('ambient')
light_datas = [[ambient_light_data]]


# Need to render an image at the ground-truth pose
d = dict()
for n in [n_refiner_iterations]:

    # Initial coarse estimate
    x = refiner_outputs[0][f'iteration={n}']
    render_img_tensor = x['renders'][0,0:3]
    render_img = tensor_image_to_uint8(render_img_tensor)
    render_img_PIL = Image.fromarray(render_img)
    render_img_PIL = adjust_brightness(render_img_PIL, factor=BRIGHTNESS_FACTOR)
    render_img_PIL.save(f'{SAVE_DIR}/refiner_iter={n}_render.png')


    img_tensor = x['images_crop'][0, 0:3]
    img = tensor_image_to_uint8(img_tensor)
    img_PIL = Image.fromarray(img)

    img_PIL.save(SAVE_DIR/f"refiner_iter={n}_img_crop.png")

    blend = BokehPlotter.make_overlay(img, np.array(render_img_PIL))

    contour_out = make_contour_overlay(img, np.array(render_img_PIL), dilate_iterations=1, color=[0,255,0])
    contour = contour_out['img']

    contour_both = make_contour_overlay(img, np.array(render_img_PIL), color=[255,0,0],
                                       dilate_iterations=0)['img']


    ### Render image at the ground-truth pose #######
    # [1,3,3]
    pred_idx = 0
    K = x['K_crop'][pred_idx].unsqueeze(0)

    df = all_infos
    df = df[df.iteration==n]

    # [1,4,4]
    TCO_gt = torch.tensor(df.iloc[0].TCO_gt).unsqueeze(0)


#     if n > 1:
#         print("TCO:\n", x['TCO_input'])
#         print("TCO_gt:\n", TCO_gt)

#     TCO_gt = x['TCO_output'][pred_idx].unsqueeze(0)
    obj_infos = [{'name': x['labels'][pred_idx]}]
#     print("obj_infos", obj_infos)



    render_out = renderer.render(labels=[object_label],
                                 TCO=TCO_gt,
                                 K=K,
                                 resolution=img.shape[:2],
                                 light_datas=light_datas)


    render_img_gt_tensor = render_out.rgbs[0]
    render_img_gt = tensor_image_to_uint8(render_img_gt_tensor)

    render_img_gt_PIL = Image.fromarray(render_img_gt)
    render_img_gt_PIL = adjust_brightness(render_img_gt_PIL, factor=BRIGHTNESS_FACTOR)
    render_img_gt_PIL.save(f'{SAVE_DIR}/refiner_iter={n}_render_gt_pose.png')


    contour_both = make_contour_overlay(contour_both, np.array(render_img_gt_PIL), color=[0,255,0],
                                       dilate_iterations=0)['img']




    contour_both_PIL = Image.fromarray(contour_both)
    contour_both_PIL.save(f'{SAVE_DIR}/refiner_iter={n}_contour_both.png')


    if data_TCO_depth_refiner is not None:
        df = data_TCO_depth_refiner.infos
        df = df[df.label == object_label]
        assert len(df) == 1, f"Found more than one prediction with label {object_label}"
        TCO = data_TCO_depth_refiner.poses[df.index.tolist()]
        render_out = renderer.render(labels=[object_label],
                                 TCO=TCO,
                                 K=K,
                                 resolution=img.shape[:2],
                                 light_datas=light_datas)
        render_img_depth_refiner_tensor = render_out.rgbs[0]
        render_img_depth_refiner = tensor_image_to_uint8(render_img_gt_tensor)
        contour_depth_refiner = make_contour_overlay(img, render_img_depth_refiner, dilate_iterations=1, color=[0,255,0])

    else:
        contour_depth_refiner = None



    d[n] = {'render': np.array(render_img_PIL),
           'img': img,
           'blend': blend,
           'contour': contour,
           'contour_out': contour_out,
           'render_gt': np.array(render_img_gt_PIL),
           'contour_both': np.array(contour_both_PIL),
            'contour_depth_refiner': contour_depth_refiner,
           }




plt.figure()
plt.imshow(d[n_refiner_iterations]['img'])
plt.show()

plt.figure()
plt.imshow(d[n_refiner_iterations]['render'])
plt.show()

plt.figure()
plt.imshow(d[n_refiner_iterations]['contour_out']['img'])
plt.title("Megapose Refiner")
plt.show()

if d[n_refiner_iterations]['contour_depth_refiner'] is not None:
    plt.figure()
    plt.imshow(d[n_refiner_iterations]['contour_depth_refiner']['img'])
    plt.title("Megapose + Depth refiner")
    plt.show()

NameError: name 'NB_DATA_DIR' is not defined

## Visualize refiner iterations

In [ ]:
print(f"scene_id: {scene_id}, view_id: {im_idx}, object_label: {object_label}")
grid = []
# plot_iter = [1,2,3,4,5,6,7,8]
plot_object_label = [object_label]
df = all_infos.copy()
df = df.loc[(df['iteration'].isin(plot_iter)) & (df['label'] == object_label)]
for _, row in df.iterrows():
    figures = row['figures']
    result_name = row['result_name']
    logit = float(row['coarse_out']['logits'][0])
    score = float(row['coarse_out']['scores'][0])
    k = row['iteration']
    figures[1].title.text = f'{result_name} / iter={k} / logit={logit:.1f}, score={score:.1f} '
    figures[2].title.text_font_size = '12pt'
    grid.append(figures)
show(gridplot(grid, sizing_mode='scale_width'))

## 3D visualization using meshcat.

Make sure you have a `meshcat-server` process running on the host machine. Otherwise this code will hang.

In [ ]:
from megapose.visualization import meshcat_utils
from megapose.visualization.meshcat_visualizer import MeshcatSceneViewer
import meshcat.geometry as g
vis = meshcat_utils.create_visualizer()

df = all_infos
show_iter = [n_refiner_iterations]
df = df.loc[(df['iteration'].isin(show_iter))]

viewer = MeshcatSceneViewer(obj_ds_name, use_textures=True,)
# viewer = MeshcatSceneViewer('ycbv', use_textures=False) # debugging
vis = viewer.visualizer

def extract_pointcloud_from_scene_data(scene_data):
    depth = scene_data.depth
    K = scene_data.camera_data.K
    pc = meshcat_utils.get_pointcloud(depth, K)

    return pc


def extract_pointclouds(iter_info):
    refiner_outputs = iter_info['refiner_outputs']
    unique_id = iter_info['refiner_instance_idx']
    image_crop_raw = refiner_outputs['images_crop_raw'][unique_id]
    render_raw = refiner_outputs['renders_raw'][unique_id]
    K_crop = refiner_outputs['K_crop'][unique_id].cpu().numpy()
    KV_crop = refiner_outputs['KV_crop'][unique_id][0].cpu().numpy()
    TCO_input = refiner_outputs['TCO_input'][unique_id].cpu().numpy()

    input_depth_dims = iter_info['input_depth_dims']
    render_depth_dims = iter_info['render_depth_dims']
    if len(input_depth_dims) > 0:
        input_depth = image_crop_raw[input_depth_dims].permute(1,2,0).cpu().squeeze().numpy()
        gt_pc = meshcat_utils.get_pointcloud(input_depth, K_crop)
    else:
        gt_pc = None


    if len(render_depth_dims) > 0:
        render_depth = render_raw[render_depth_dims].permute(1,2,0).cpu().squeeze().numpy()
        render_pc = meshcat_utils.get_pointcloud(render_depth, KV_crop)
    else:
        render_pc = None


    return {'gt': gt_pc,
           'render': render_pc,
            'TCO_input': TCO_input,
           }

def plot_results(df):
    df = df.to_dict('records')

    obj_infos = []
    TCO_gt = df[0]['TCO_gt']
    TOC_gt = np.linalg.inv(TCO_gt)
    obj_label = df[0]['label']
    # Visualize camera

    for row in df:
        meshcat_prefix = f"{row['result_name']}"
        TOgt_O = np.linalg.inv(TCO_gt) @ row['TCO_input']
        k = f"{row['result_name']}/iteration={row['iteration']}/mesh"
        obj_infos.append({'name': obj_label, 'TWO': TOgt_O, 'node_name': k})


    if data_TCO_depth_refiner is not None:
        df_ = data_TCO_depth_refiner.infos
        idx = df_[df_.label == object_label].iloc[0].name
        TCO_depth_refiner = data_TCO_depth_refiner.poses[idx].cpu().numpy()
        TOgt_O = np.linalg.inv(TCO_gt) @ TCO_depth_refiner
        k = f"{row['result_name']}/depth_refiner/mesh"
        obj_infos.append({'name': obj_label, 'TWO': TOgt_O, 'node_name': k})


#     if 'TCO_coarse_init' in df[0]:
#         TCO_coarse_init = cast_to_numpy(df[0]['TCO_coarse_init'], np.float64)
#         TOgt_O = TOC_gt @ TCO_coarse_init
#         obj_infos.append({'name': obj_label, 'TWO': TOgt_O, 'node_name': f'{meshcat_prefix}/TCO_coarse_init/mesh'})


    obj_infos.append({'name': obj_label, 'TWO': np.eye(4), 'node_name': 'ground_truth'})
    viewer.visualize_scene(obj_infos)

    # Extra visualization must be after the 'visualize_scene' call
    meshcat_utils.make_frame(vis, "camera", transform=TOC_gt, ignore_invalid_transform=True)

    # Line connecting camera and origin
    vertices = np.zeros([3,2])
    vertices[:, 1] = TOC_gt[:3,3]
    vis['line'].set_object(g.Line(g.PointsGeometry(vertices)))


    # visualize ground-truth pointcloud
    pc_gt = extract_pointcloud_from_scene_data(df[0]['scene_data'])
    meshcat_utils.visualize_pointcloud(vis, 'ground_truth_pointcloud', pc_gt, transform=TOC_gt,
                                              color=[0,255,0])


    # visualize depth images, but only for final index
    for row in df:
        pc_data = extract_pointclouds(row)
        f"{row['result_name']}/iteration={row['iteration']}"
#         if pc_data['gt'] is not None:
#             print("Visualizing ground-truth pointcloud")
#             meshcat_utils.visualize_pointcloud(vis, 'ground_truth_pointcloud', pc_data['gt'], transform=TOC_gt,
#                                               color=[0,255,0])

        if pc_data['render'] is not None:
            TOgt_O = np.linalg.inv(TCO_gt) @ row['TCO_input']
            TCO_input = pc_data['TCO_input']
            TOC_input = np.linalg.inv(TCO_input)
            TOgt_input = TOgt_O @ TOC_input # transform rendered to observed

            k = f"{row['result_name']}/iteration={row['iteration']}/pointcloud"
            meshcat_utils.visualize_pointcloud(vis, k, pc_data['render'],
                                               transform=TOC_gt, color=[255,255,0])

    return

plot_results(df)

## Print accuracy data

In [ ]:
df = all_infos
df_first_iter = df[df.iteration==1]
trans_err_init = float(df_first_iter.trans_err)
rot_err_init = float(df_first_iter.rot_err)
last_iter = df.iteration.max()
# print("last_iter", last_iter)
df_final_iter = df[df.iteration==df.iteration.max()]

trans_err = float(df_final_iter.trans_err)
rot_err = float(df_final_iter.rot_err)
# print(df_final_iter.tran)
# df_final_iter = df_final_iter.iloc[0]

SO3_grid_size = pose_estimator._SO3_grid.shape[0]
# print(x_failure)
print(f"result_name: {result_name}, SO3-grid-size={SO3_grid_size}")
print(f"iteration={last_iter}")
print(f"per_iter_depth_multiplier:", refiner_model.per_iter_depth_multiplier)
print("depth_multiplier:", refiner_model.depth_multiplier)
print(f"\ninitial translation error (cm): {trans_err_init*100:.2f}")
print(f"initial rot_err (no sym) (deg): {rot_err_init:.1f}")
print(f"\ntranslation error (cm): {trans_err*100:.2f}\nrot_err (no sym) (deg): {rot_err:.1f}")

#####































